<a href="https://colab.research.google.com/github/NehalH/RupeeClassifier_MobileNet/blob/main/RupeeClassifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define an ImageDataGenerator for training data
train_datagen = ImageDataGenerator(rescale=1./255)

# Specify the path to your dataset on Google Drive
dataset_path = '/content/drive/MyDrive/A-eye/Datasets/Indian Currency Dataset'

# Manually specify the classes
class_names = ['10', '20', '50', '100', '200']

# Create a data generator for training images with manual class specification
train_generator = train_datagen.flow_from_directory(
    dataset_path,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    classes=class_names  # Manually specify the class names
)

Found 1269 images belonging to 5 classes.


In [3]:
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D

# Load MobileNet base model (pre-trained on ImageNet)
base_model = MobileNet(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Build your custom model on top of MobileNet
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(128, activation='relu'),
    Dense(len(class_names), activation='softmax')
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model on the training data
model.fit(train_generator, epochs=10)  # Adjust the number of epochs based on your requirements

Epoch 1/10
40/40 [==============================] - 224s 5s/step - loss: 0.4688 - accuracy: 0.8392
Epoch 2/10
40/40 [==============================] - 201s 5s/step - loss: 0.1113 - accuracy: 0.9638
Epoch 3/10
40/40 [==============================] - 197s 5s/step - loss: 0.0332 - accuracy: 0.9882
Epoch 4/10
40/40 [==============================] - 201s 5s/step - loss: 0.0678 - accuracy: 0.9827
Epoch 5/10
40/40 [==============================] - 202s 5s/step - loss: 0.0745 - accuracy: 0.9756
Epoch 6/10
40/40 [==============================] - 204s 5s/step - loss: 0.0611 - accuracy: 0.9858
Epoch 7/10
40/40 [==============================] - 217s 5s/step - loss: 0.0743 - accuracy: 0.9835
Epoch 8/10
40/40 [==============================] - 213s 5s/step - loss: 0.0742 - accuracy: 0.9724
Epoch 9/10
40/40 [==============================] - 213s 5s/step - loss: 0.0485 - accuracy: 0.9835
Epoch 10/10
40/40 [==============================] - 194s 5s/step - loss: 0.0776 - accuracy: 0.9811


In [6]:
import tensorflow as tf

# Convert the model to TensorFlow Lite
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the TFLite model
with open('/content/drive/MyDrive/A-eye/Models/rupee_classifier_mobilenet_model.tflite', 'wb') as f:
    f.write(tflite_model)
